In [42]:
import numpy as np

import CAS.dmrghandler.src.dmrghandler as dmrghandler
from CAS.dmrghandler.src.dmrghandler import dmrg_calc_prepare
from pathlib import Path
import scipy
import pandas as pd
import os
import itertools
import xlsxwriter

### Spatial to Spin orbitals transformations

In [46]:
def orb_to_spin_orbital_obt(obt):
    n = obt.shape[0]
    obt_spin = np.zeros((2 * n, 2 * n))
    for p in range(n):
        for q in range(n):
            obt_spin[2 * p, 2 * q] = obt[p, q]
            obt_spin[2 * p + 1, 2 * q + 1] = obt[p, q]
    
    return obt_spin

def otb_to_spin_orbital_tbt(tbt):
    n = tbt.shape[0]
    tbt_spin = np.zeros((2 * n, 2 * n, 2 * n, 2 * n))
    for p in range(n):
        for q in range(n):
            for r in range(n):
                for s in range(n):
                    tbt_spin[2 * p, 2 * q, 2 * r, 2 * s] = tbt[p, q, r, s]
                    tbt_spin[2 * p + 1, 2 * q + 1, 2 * r, 2 * s] = tbt[p, q, r, s]
                    tbt_spin[2 * p, 2 * q, 2 * r + 1, 2 * s + 1] = tbt[p, q, r, s]
                    tbt_spin[2 * p + 1, 2 * q + 1, 2 * r + 1, 2 * s + 1] = tbt[p, q, r, s]
    
    return tbt_spin

def otb_to_spin_orbital(tensor):
    if len(tensor.shape) == 2:
        return orb_to_spin_orbital_obt(tensor)
    elif len(tensor.shape) == 4:
        return otb_to_spin_orbital_tbt(tensor)

### Hubbard terms definitions

In [49]:
def hubbard_epsilon(obt_spatial):
    n = obt_spatial.shape[0]
    total = 0
    for i in range(n):
        total += obt_spatial[i, i]
    return total / n

def hubbard_negative_t(obt_spatial):
    n = obt_spatial.shape[0]
    total = 0
    for p in range(1, n-1):
        total += obt_spatial[p - 1, p]
        total += obt_spatial[p, p + 1]
    return total / (2 * (n - 1))

def hubbard_u(tbt_spatial):
    n = tbt_spatial.shape[0]
    total = 0
    for p in range(n):
        total += tbt_spatial[p, p, p, p]
    return total / n

In [101]:
def hubbard_epsilon_term(obt_spatial, obt_spin):
    nspin = obt_spin.shape[0]
    norb = obt_spin.shape[0] // 2
    epsilon_term = np.zeros((nspin, nspin))
    epsilon = hubbard_epsilon(obt_spatial)
    for i in range(norb):
        epsilon_term[2 * i, 2 * i] = epsilon
        epsilon_term[2 * i + 1, 2 * i + 1] = epsilon
    return epsilon_term

def hubbard_t_term(obt_spatial, obt_spin):
    nspin = obt_spin.shape[0]
    norb = obt_spin.shape[0] // 2
    t_term = np.zeros((nspin, nspin))
    t = hubbard_negative_t(obt_spatial)
    for i in range(norb-1):
        j = i + 1
        t_term[2 * j, 2 * i] = t
        t_term[2 * j + 1, 2 * i + 1] = t
        t_term[2 * i, 2 * j] = t
        t_term[2 * i + 1, 2 * j + 1] = t
    return t_term

def hubbard_u_term(tbt_spatial, tbt_spin):
    nspin = tbt_spin.shape[0]
    norb = tbt_spin.shape[0] // 2
    u_term = np.zeros((nspin, nspin, nspin, nspin))
    u = hubbard_u(tbt_spatial)
    for i in range(norb):
        u_term[2 * i, 2 * i, 2 * i + 1, 2 * i + 1] = u
    
    return u_term


### Tests for Hubbard model Hamiltonian construction

In [ ]:
def test_epsilon_term(epsilon_term):
    indices = [i for i in range(epsilon_term.shape[0])]
    for pair in itertools.product(indices, indices):
        if pair[0] != pair[1]:
            assert epsilon_term[pair[0], pair[1]] == 0, "epsilon term not correct symmetry"

def test_t_term(t_term):
    indices = [i for i in range(t_term.shape[0])]
    for pair in itertools.product(indices, indices):
        if pair[0] == pair[1]:
            assert t_term[pair[0], pair[1]] == 0, "t term not correct symmetry"
        elif pair[0] == pair[1] + 1:
            assert t_term[pair[0], pair[1]] == t_term[pair[1], pair[0]], "t term not correct symmetry"

def test_U_term(U_term):
    indices = [i for i in range(U_term.shape[0])]
    for pair in itertools.product(indices, indices, indices, indices):
        if pair[0] % 2 == 0 and  pair[0] == pair[1] and pair[2] == pair[0] + 1 and pair[2] == pair[3]:
            assert U_term[pair[0], pair[1], pair[2], pair[3]] != 0, "U term not correct symmetry"
        else:
            assert U_term[pair[0], pair[1], pair[2], pair[3]] == 0, "U term not correct symmetry"

### Iterate through the FCIDUMP file directory

In [104]:
fcidump_path =Path("small_catalysts")
# Create Excel writer object
writer = pd.ExcelWriter("nearness_to_hubbard_june4.xlsx", engine='xlsxwriter')

In [105]:

column_names = ["FCIDUMP file", "L2 norm difference for obt", "L2 norm difference for tbt", "F"]
data_to_be_added = []
for filename in os.listdir(fcidump_path):
    print(f"Current file: {filename}")
    (
    one_body_tensor,
    two_body_tensor,
    nuc_rep_energy,
    num_orbitals,
    num_spin_orbitals,
    num_electrons,
    two_S,
    two_Sz,
    orb_sym,
    extra_attributes,
    ) = dmrg_calc_prepare.load_tensors_from_fcidump(data_file_path=Path(fcidump_path)/Path(filename), molpro_orbsym_convention=True)
    
    obt_spin = otb_to_spin_orbital(one_body_tensor)
    tbt_spin = otb_to_spin_orbital(two_body_tensor)
    print("Orb to spin done")
    epsilon_term = hubbard_epsilon_term(one_body_tensor.copy(), obt_spin.copy()) 
    test_epsilon_term(epsilon_term)
    t_term = hubbard_t_term(one_body_tensor.copy(), obt_spin.copy())
    test_t_term(t_term)
    H_hubbard_obt = epsilon_term + t_term
    print("Hubbard one body done")
    U_term = hubbard_u_term(two_body_tensor.copy(), tbt_spin.copy())
    test_U_term(U_term)
    H_hubbard_tbt = U_term
    
    
    print("Hubbard done")
    l2_diff_obt = scipy.linalg.norm(obt_spin - H_hubbard_obt, ord=None)
    l2_diff_tbt = scipy.linalg.norm(tbt_spin - H_hubbard_tbt, ord=None)
    
    F = l2_diff_obt + l2_diff_tbt
    print(f"FCIDUMP file: {filename}")
    print(f"L2 norm difference for obt: {l2_diff_obt}")
    print(f"L2 norm difference for tbt: {l2_diff_tbt}")
    print(f"F: {F}")
    data_to_be_added.append([filename, l2_diff_obt, l2_diff_tbt, F])


Current file: fcidump.2_co2_6-311++G__
Parsing small_catalysts/fcidump.2_co2_6-311++G__
Orb to spin done
Hubbard one body done
Hubbard done
FCIDUMP file: fcidump.2_co2_6-311++G__
L2 norm difference for obt: 3.208107750755828
L2 norm difference for tbt: 6.411064467593759
F: 9.619172218349586


In [106]:
df_1 = pd.DataFrame()
#Iteratively add data to new columns
for column_name, column_data in zip(column_names, data_to_be_added):
    df_1[column_name] = column_data

df_1.to_excel(writer, sheet_name="Hubbard distance result", index=False)

In [107]:
correct = []
column1 = []
column2 = []
column3 = []
column4 = []
for q in data_to_be_added:
    column1.append(q[0])
    column2.append(q[1])
    column3.append(q[2])
    column4.append(q[3])

In [108]:
correct.append(column1)
correct.append(column2)
correct.append(column3)
correct.append(column4)

In [109]:
print(correct)

[['fcidump.2_co2_6-311++G__'], [3.208107750755828], [6.411064467593759], [9.619172218349586]]
